Question 1
Install Pipenv
What's the version of pipenv you installed?
Use --version to find out

In [ ]:
pipenv --version
pipenv, version 2021.5.29

Question 2
Use Pipenv to install Scikit-Learn version 1.0
What's the first hash for scikit-learn you get in Pipfile.lock?

In [ ]:
"sha256:121f78d6564000dc5e968394f45aac87981fcaaf2be40cfcd8f07b2baa1e1829"

Question 3
Let's use these models!

Write a script for loading these models with pickle
Score this customer:
{"contract": "two_year", "tenure": 12, "monthlycharges": 19.7}

In [1]:
import pickle

with open('dv.bin','rb') as f_dv_in:
    dv=pickle.load(f_dv_in)
    

with open('model1.bin','rb') as f_dv_in:
    model=pickle.load(f_dv_in)

customer={"contract": "two_year", "tenure": 12, "monthlycharges": 19.7}

X=dv.transform([customer])

model.predict_proba(X)[0,1]

#0.11549580587832914

c:\program files\python37\lib\site-packages\sklearn\base.py:315: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.0 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\program files\python37\lib\site-packages\sklearn\base.py:315: UserWarning: Trying to unpickle estimator LogisticRegression from version 1.0 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


0.11549580587832914

Dummy Flask app

In [ ]:
from flask import Flask

app=Flask('ping')

@app.route('/ping', methods=['GET'])
def ping():
    return 'hello'

if __name__=="__main__":
    app.run(debug=True, host='0.0.0.0', port=9090,use_reloader=False)
    #app.run(debug=True, host='0.0.0.0', port=9090,use_reloader=False)

 * Serving Flask app 'ping' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on all addresses.
 * Running on http://172.26.3.239:9090/ (Press CTRL+C to quit)
172.26.3.239 - - [11/Oct/2021 10:39:02] "GET / HTTP/1.1" 404 -
172.26.3.239 - - [11/Oct/2021 10:39:03] "GET /favicon.ico HTTP/1.1" 404 -
172.26.3.239 - - [11/Oct/2021 10:39:06] "GET /ping HTTP/1.1" 200 -


Question 4
Now let's serve this model as a web service

Install Flask and Gunicorn (or waitress, if you're on Windows)
Write Flask code for serving the model
Now score this customer using requests:
url = "YOUR_URL"
customer = {"contract": "two_year", "tenure": 1, "monthlycharges": 10}
requests.post(url, json=customer).json()
What's the probability that this customer is churning?

In [ ]:
from flask import Flask, request, jsonify

app=Flask('churn')

@app.route('/test',methods=['POST'])

def predict():
    customer=request.get_json()
    X = dv.transform([customer])
    y_pred = model.predict_proba(X)[0,1]
    churn=y_pred>=0.5
    result={ 'churn_probability': float(y_pred), 'churn': bool(churn) }
    return jsonify(result)

if __name__=="__main__":
    app.run(debug=True, host='0.0.0.0', port=9096,use_reloader=False)


 * Serving Flask app 'churn' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on all addresses.
 * Running on http://172.26.3.239:9096/ (Press CTRL+C to quit)
172.26.3.239 - - [11/Oct/2021 10:50:06] "POST /test HTTP/1.1" 200 -


Save the above code in as py and run this 
waitress-serve --listen=0.0.0.0:9696 predict:app

Run the below code after running the above line from a seperate py file

In [3]:
# Run this from seperate note book or python file
import requests
url='http://localhost:9696/predict'
customer = {"contract": "two_year", "tenure": 1, "monthlycharges": 10}
requests.post(url, json=customer).json()

#{'churn': True, 'churn_probability': 0.9988892771007961}

{'churn': True, 'churn_probability': 0.9988892771007961}

Question 5
Now create your own Dockerfile based on the image I prepared.

It should start like that:
Alternatively, you can get this information when running docker images - it'll be in the "IMAGE ID" column. Submitting DIGEST (long string starting with "sha256") is also fine.

In [ ]:
sha256:02d4fd53bfaf744018213731f34ca17c9e996c23a6e1680fb1ec4ae33731aad5

Question 6
Let's run your docker container!

After running it, score this customer:

In [ ]:
url = "http://localhost:9696/predict"
customer = {"contract": "two_year", "tenure": 12, "monthlycharges": 10}
requests.post(url, json=customer).json()
#{'churn': True, 'churn_probability': 0.7284944888182928}

Docker file

FROM agrigorev/zoomcamp-model:3.8.12-slim                                              
RUN pip install pipenv                                                 
WORKDIR /app                                                           
COPY ["Pipfile", "Pipfile.lock", "./"]                                 
RUN pipenv install --deploy --system                                   
COPY ["*.py","./"] 
RUN pip install gunicorn                                 
EXPOSE 9696                                                            
ENTRYPOINT ["gunicorn", "--bind", "0.0.0.0:9696", "churn_serving:app"]

--Checking ingnore
import pickle

with open('app/dv.bin','rb') as f_dv_in:
    dv=pickle.load(f_dv_in)
    

with open('app/model2.bin','rb') as f_dv_in:
    model=pickle.load(f_dv_in)

customer={"contract": "two_year", "tenure": 12, "monthlycharges": 10}

X=dv.transform([customer])

model.predict_proba(X)[0,1]
#0.7284944888182928